In [0]:
print ('Hello')

Hello


In [0]:
!pip install -U git+git://github.com/Computational-Content-Analysis-2020/lucem_illud_2020.git
import lucem_illud_2020 #pip install -U git+git://github.com/Computational-Content-Analysis-2020/lucem_illud_2020.git
import pandas #For DataFrames
import numpy as np #For arrays
import pickle #if you want to save layouts
import os
import re
import zipfile
import sys
import spacy
nlp=spacy.load('en_core_web_sm')

  Cloning git://github.com/Computational-Content-Analysis-2020/lucem_illud_2020.git to /tmp/pip-req-build-wovhf220
  Running command git clone -q git://github.com/Computational-Content-Analysis-2020/lucem_illud_2020.git /tmp/pip-req-build-wovhf220
     |████████████████████████████████| 5.5MB 2.8MB/s 
     |████████████████████████████████| 122kB 50.6MB/s 
     |████████████████████████████████| 460kB 45.1MB/s 
     |████████████████████████████████| 81kB 10.9MB/s 
     |████████████████████████████████| 32.8MB 117kB/s 
     |████████████████████████████████| 481kB 55.9MB/s 
     |████████████████████████████████| 71kB 11.1MB/s 
     |████████████████████████████████| 450kB 49.8MB/s 
     |████████████████████████████████| 3.1MB 39.4MB/s 
     |████████████████████████████████| 870kB 54.8MB/s 
     |████████████████████████████████| 1.0MB 46.8MB/s 
  Created wheel for lucem-illud-2020: filename=lucem_illud_2020-8.0.1-cp36-none-any.whl size=35151 sha256=023058b9efec564ed97dafe8fbda040d9

In [0]:
#final
def loadDavies(address, corpus_style="text", num_files=10000, return_raw=False):
    texts_raw = {}
    article_count=0
    for file in os.listdir(address + "/"):
        #change text name to month of interest
        if file.startswith('text_16-10-ykw.zip'):
            if corpus_style in file:
                print(type(file),file)
                zfile = zipfile.ZipFile(address + "/" + file)
                for file in zfile.namelist():
                    #check for us source
                    if file.lower().find('us')==-1:
                        continue
                    texts_raw[file] = []
                    with zfile.open(file) as f:
                        for article in f:
                            article_lower = article.decode().lower()
                            #only take articles with election in them
                            finde = 'election' in article_lower
                            if (finde):
                                article_count+=1
                                texts_raw[file].append(article)
    print (article_count)                            
    if return_raw:
        return(texts_raw)

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#corpus_name = "/project2/soci40133/Davies_corpora/NOW"
#corpus_name='/content/gdrive/My Drive/SOCI 40133 - Final Project/NOW/'

censored_df=pandas.read_csv('/content/gdrive/My Drive/SOCI 40133 - Final Project/Data/Censored(archive)/censored.csv')
censored_df.head()
#news_raw_16_10 = loadDavies(corpus_name, corpus_style="text", num_files=10000, return_raw=True)
#news_raw = loadDavies(corpus_name, corpus_style="text", num_files=10000, return_raw=True)


,link,date,headline,source,text
0,https://theantimedia.com/mike-pompeo-wants-mid...,11-Oct-18,Mike Pompeo: The US Wants the Entire Middle Ea...,"('MEMO', 'https://www.middleeastmonitor.com/20...",(MEMO) — US Secretary of State Mike Pompeo sai...
1,https://theantimedia.com/pressure-saudi-arabia...,11-Oct-18,Trump Pressures Saudi Arabia to Explain What H...,"('MEMO', 'https://www.middleeastmonitor.com/20...",(MEMO) — US President Donald Trump increased p...
2,https://theantimedia.com/western-democracy-bra...,9-Oct-18,Future of Western Democracy Being Played Out i...,"('CN', 'https://consortiumnews.com/2018/10/09/...","Stripped to its essence, the Brazilian preside..."
3,https://theantimedia.com/israel-shoot-to-kill-...,9-Oct-18,Israel’s Education Minister Wants “Shoot to Ki...,"('MEMO', 'https://www.middleeastmonitor.com/20...",(MEMO) — Israel’s Education Minister Naftali B...
4,https://theantimedia.com/nikki-haley-resigns-u...,9-Oct-18,Nikki Haley Resigns as UN Ambassador,"('ZHE', 'https://www.zerohedge.com/news/2018-1...",(ZHE) — Update (11:00 am ET): During their jo...


In [0]:
censored_df.rename(columns={"link": "url", "headline": "title"})
print(censored_df.shape)
censored_df=censored_df.drop([601])
print(censored_df.shape)
# for x in range(censored_df.shape[0]):
#   try:
#     censored_df.loc[x]['text'].encode
#   except:
#     print(x)
censored_df['text']=[x.encode() for x in censored_df['text']]
censored_df.to_csv('/content/gdrive/My Drive/SOCI 40133 - Final Project/Data/Censored(archive)/censored_processed.csv')

(791, 5)
(790, 5)


In [0]:
source_list=list(censored_df['source'])
#source_list= [s.replace(' ',' Opinion') for s in source_list]
source_list_new=['('+x.split(',')[0].replace('(','').replace('\'','')+')' for x in source_list]
replace_list =(list(set(source_list_new).difference(set(['(explained)','(ANTIMEDIA\\xa0)','(Rather)','(UNEARTHED)','(gave a speech)']))))
print (replace_list)

['(CD )', '(SC )', '(FEE )', '(FPIF)', '(TIM)', '(CN )', '(retracted)', '(CR)', '(CD)', '(MPN)', '(PROPUBLICA)', '(CONVERSATION)', '(AHT)', '(FEE)', '(ANTIWAR)', '(TRUEACTIVIST)', '(COMMONDREAMS)', '(OD )', '(TFC )', '(INTERCEPT)', '(MEE)', '(AP)', '(FAIR)', '(CONSORTIUMNEWS)', '(MEMO )', '(CN)', '(ZH)', '(SP)', '(ACTIVISTPOST)', '(TFC)', '(GPA)', '(MEMO)', '(TMU)', '(RN)', '(RPI)', '(ZHE )', '(OD)', '(ZHE)', '(ANTIWAR.COM)', '(SHADOWPROOF)', '(CJ)', '(PP)', '(ANTIMEDIA)']


In [0]:
# for s in source_list:
#   if 'OD' in s:
#     print(s)
#'('+"('OD Op-ed', 'https://www.middleeastmonitor.com/20".split(',')[0].replace('(','').replace('\'','')+')'

In [0]:
#TODO:get rid of everything before the first p tag and store headline seperately
def clean_raw_text(raw_articles):
    clean_articles = []
    for i in range(len(raw_articles)):
        try:
            text = raw_articles[i]
            text = text.decode("utf-8").lower()
            #gavin macfayden, Saeb Erekat, fake news, citizen united, super PAC, donald trump

            clean_article = text.replace(" \'m", "'m").replace(" \'ll", "'ll").replace(" \'re", "'re").replace(" \'s", "'s").replace(" \'re", "'re").replace(" n\'t", "n't").replace(" \'ve", "'ve").replace(" /'d", "'d")\
                            .replace("<h>","").replace("<p>","").replace("@@","").replace("n’t","n't").replace("’","'")
            for r in replace_list:
              clean_article = clean_article.replace(r.lower(),'')
            clean_articles.append(clean_article)
        except AttributeError:
            print("ERROR CLEANING")
            print(censored_df.loc[i])
            print(i)
            continue
        except UnicodeDecodeError:
            print("Unicode Error, Skip")
            continue
    return clean_articles

In [0]:
def word_tokenize(text, remove_stop_words=False):
    tokenized = []
    customize_stop_words = ['is']
    # pass word list through language model.
    doc = nlp(text)
    
    for w in customize_stop_words:
        nlp.vocab[w].is_stop = True
    for token in doc:
        if not token.is_punct and len(token.text.strip()) > 0:
            if remove_stop_words:
                if not token.is_stop: 
                   tokenized.append(token.text)
                else:
                    continue
            else:
                tokenized.append(token.text)
    
    return tokenized

In [0]:
def help(sentences):
  n_sentences = []
  flag = 0
  for sent in sentences:
    n_sentence = []
    prev= ''
    for token in sent:
      if token.isalpha():
        if token in ['donald','john','citizen','gavin','saeb','fake','super','election','super']:
          prev = token
          flag=1
          continue
        else:
          if token == 'trump' and prev =='donald':
            n_sentence.append('donald_trump')
            flag = 0
          elif token == 'pilger' and prev =='john':
            n_sentence.append('john_pilger') 
            flag = 0
          elif token == 'unite' and prev =='citizen':
            n_sentence.append('citizens_united') 
            flag = 0
          elif token == 'saeb' and prev =='erekat':
            n_sentence.append('saeb_erekat')
            flag = 0
          elif token == 'news' and prev =='fake':
            n_sentence.append('fake_news')
            flag = 0
          elif token == 'meddle' and prev =='election':
            n_sentence.append('election_meddling') 
            flag = 0
          elif token == 'interference' and prev =='election':
            n_sentence.append('election_interference') 
            flag = 0
          elif token == 'pac' and prev =='super':
            n_sentence.append('super_pac')
            flag = 0
          elif flag==1:
              n_sentence.append(prev)
              flag = 0
              n_sentence.append(token)  
          else:
              n_sentence.append(token)
          prev = token
    if (n_sentence!=[]):
      n_sentences.append(n_sentence)
  return n_sentences 


In [0]:
import time

t0 = time.time()
ta = time.time()
a_count =0
news_texts = {}
just_texts = {}

#article=news_raw[file]
print('No of censored articles', len(list(censored_df['text'])))
articles = clean_raw_text(list(censored_df['text']))


for article in articles:
  a_count+=1
  if (a_count%100)==0:
      t=time.time()
      print(t-ta)
      ta=time.time()
      print('Processing',a_count)

  ##where we sentence break, tokenize sentences, flatten, etc.
  tokenized_sents = [lucem_illud_2020.word_tokenize(s) for s in lucem_illud_2020.sent_tokenize(article)]
  
  normalized_sents = [lucem_illud_2020.normalizeTokens(s) for s in tokenized_sents]
  
  sents = []
  for s in normalized_sents:
    t=[x.replace('medium','media') for x in s]
    sents.append(t)
  sents = help(sents)
  normalized_sents = sents
    
  #remove empty lists
  normalized_sents =  list(filter(None, normalized_sents))
  #print(normalized_sents)
  tokens = [item for sublist in normalized_sents for item in sublist]
  tid=article.split(" ")[0]
  #print(tid)
  #print(tokens)
  #tokens_spacy= word_tokenize(article)
  try:
      news_texts[a_count] = (tokens,normalized_sents)
      just_texts[a_count] = article
  except IndexError:
      print("Missing: ",txts[0][2:])
      continue
t1 = time.time()
print ("Total time", t1-t0)   
assert(len(list(news_texts.keys()))==a_count) 

No of censored articles 790
18.900840044021606
Processing 100
17.20031237602234
Processing 200
13.832902669906616
Processing 300
14.997545719146729
Processing 400


KeyboardInterrupt: ignored

In [0]:
article_cleaned=articles[0]
print(article_cleaned)

 — us secretary of state mike pompeo said yesterday that “israel is everything we want the entire middle east to look like going forward” and that us-israel relations are “stronger than ever”. speaking at an award ceremony held by the jewish institute for national security of america in washington dc, pompeo hailed israel as “democratic and prosperous”, adding “it desires peace, it is a home to a free press and a thriving economy,” arutz sheva reported. pompeo then compared israel to iran, citing the latter's “corrupt leaders [who] assault the human rights of their own people and finance terrorism in every corner of the middle east”. pompeo lamented that the us administration under former president barack obama “had more respect for the leaders of iran than it did for the state of israel,” stressing that “under president trump's leadership, the united states is standing where it should firmly be: on the side of israel.” pompeo also discussed the long-awaited us peace initiative – dubbe

In [0]:
# for r in replace_list:
#               clean_article = text.replace(r,'')

In [0]:
ids=set((news_texts.keys()))
len(ids)

425

In [0]:
# source matching
sources = []
try:
  zfile = zipfile.ZipFile(corpus_name + "/censored.zip")

  for file in zfile.namelist():
    with zfile.open(file) as f:
        for line in f:
            sources.append(line)
except:
  print('Source file not found')
  print('Using empty source')          

Source file not found
Using empty source


In [0]:
sl=[]
ecount=0
for s in sources:
    try:
        sl.append(s.decode().split("\t")[0].strip())
        #print(s.decode().split("\t")[2].strip())
    except UnicodeDecodeError:
        ecount+=1
        continue
    

In [0]:
print("Total source documents", len(sources))
print("Files with decode errors:", ecount)
print("Files whose sources intersect with the files we have collected", len(set(sl).intersection(ids)))
print ("Total files", len(ids))

Total source documents 0
Files with decode errors: 0
Files whose sources intersect with the files we have collected 0
Total files 790


In [0]:
#news_texts[list(set(sl).intersection(ids))[0]]

In [0]:
#the tids not found in sources
censored_df['tokens']=[news_texts[tids][0] for tids in list(news_texts.keys())]
censored_df['normalized_tokens']=[news_texts[tid][1] for tid in list(news_texts.keys())]


In [0]:
censored_df.shape

(790, 7)

In [0]:
censored_df

,link,date,headline,source,text,tokens,normalized_tokens
0,https://theantimedia.com/mike-pompeo-wants-mid...,11-Oct-18,Mike Pompeo: The US Wants the Entire Middle Ea...,"('MEMO', 'https://www.middleeastmonitor.com/20...",b'(MEMO)\xc2\xa0\xe2\x80\x94\xc2\xa0US Secreta...,"[secretary, state, mike, pompeo, say, yesterda...","[[secretary, state, mike, pompeo, say, yesterd..."
1,https://theantimedia.com/pressure-saudi-arabia...,11-Oct-18,Trump Pressures Saudi Arabia to Explain What H...,"('MEMO', 'https://www.middleeastmonitor.com/20...",b'(MEMO)\xc2\xa0\xe2\x80\x94\xc2\xa0US Preside...,"[president, donald_trump, increase, pressure, ...","[[president, donald_trump, increase, pressure,..."
2,https://theantimedia.com/western-democracy-bra...,9-Oct-18,Future of Western Democracy Being Played Out i...,"('CN', 'https://consortiumnews.com/2018/10/09/...","b'Stripped to its essence, the Brazilian presi...","[strip, essence, brazilian, presidential, elec...","[[strip, essence, brazilian, presidential, ele..."
3,https://theantimedia.com/israel-shoot-to-kill-...,9-Oct-18,Israel’s Education Minister Wants “Shoot to Ki...,"('MEMO', 'https://www.middleeastmonitor.com/20...","b""(MEMO)\xc2\xa0\xe2\x80\x94\xc2\xa0Israel\xe2...","[israel, education, minister, naftali, bennett...","[[israel, education, minister, naftali, bennet..."
4,https://theantimedia.com/nikki-haley-resigns-u...,9-Oct-18,Nikki Haley Resigns as UN Ambassador,"('ZHE', 'https://www.zerohedge.com/news/2018-1...",b'(ZHE)\xc2\xa0\xe2\x80\x94\xc2\xa0 Update (11...,"[update, et, joint, press, conference, trump, ...","[[update, et, joint, press, conference, trump,..."
...,...,...,...,...,...,...,...
786,https://theantimedia.com/half-billion-fake-pro...,3-Oct-16,The Pentagon Spent Half a Billion Dollars Crea...,"('ANTIMEDIA', 'https://theantimedia.org/')",b'(ANTIMEDIA) The Department of Defense spent ...,"[department, defense, spend, half, dollar, fun...","[[department, defense, spend, half, dollar, fu..."
787,https://theantimedia.com/hacked-audio-hillary-...,3-Oct-16,Hacked Audio Reveals What Hillary Really Think...,"('ZHE', 'http://www.zerohedge.com/news/2016-09...",b'(ZHE)\xc2\xa0The reason why the Trump campai...,"[reason, trump, campaign, eager, find, transcr...","[[reason, trump, campaign, eager, find, transc..."
788,https://theantimedia.com/democrat-registered-1...,1-Oct-16,Meet The Democrat Who Registered 19 Dead Peopl...,"('ZHE', 'http://www.zerohedge.com/news/2016-09...",b'(ZHE)\xc2\xa0Just yesterday Zero Hedge\xc2\x...,"[yesterday, hedge, write, fbi, investigation, ...","[[yesterday, hedge, write, fbi, investigation,..."
789,https://theantimedia.com/chelsea-clinton-went-...,29-Sep-16,Chelsea Clinton Just Went Full Anti-Science,"('ANTIMEDIA', 'https://theantimedia.org/')",b'(ANTIMEDIA)\xc2\xa0Chelsea Clinton has gone ...,"[chelsea, clinton, go, anti, science, big, way...","[[chelsea, clinton, go, anti, science, big, wa..."


In [0]:
#sanity check
#news_df.loc['14637217']['tokens'][:100]

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
censored_df.to_pickle('/content/drive/My Drive/SOCI 40133 - Final Project/Data/censored_df_new.pkl')

In [0]:
censored_df=pandas.read_pickle('/content/gdrive/My Drive/SOCI 40133 - Final Project/Data/censored_df.pkl')

In [0]:
censored_df.head()


,link,date,headline,source,text,tokens,normalized_tokens
0,https://theantimedia.com/mike-pompeo-wants-mid...,11-Oct-18,Mike Pompeo: The US Wants the Entire Middle Ea...,"('MEMO', 'https://www.middleeastmonitor.com/20...",b'(MEMO)\xc2\xa0\xe2\x80\x94\xc2\xa0US Secreta...,"[secretary, state, mike, pompeo, say, yesterda...","[[secretary, state, mike, pompeo, say, yesterd..."
1,https://theantimedia.com/pressure-saudi-arabia...,11-Oct-18,Trump Pressures Saudi Arabia to Explain What H...,"('MEMO', 'https://www.middleeastmonitor.com/20...",b'(MEMO)\xc2\xa0\xe2\x80\x94\xc2\xa0US Preside...,"[president, donald, trump, increase, pressure,...","[[president, donald, trump, increase, pressure..."
2,https://theantimedia.com/western-democracy-bra...,9-Oct-18,Future of Western Democracy Being Played Out i...,"('CN', 'https://consortiumnews.com/2018/10/09/...","b'Stripped to its essence, the Brazilian presi...","[strip, essence, brazilian, presidential, elec...","[[strip, essence, brazilian, presidential, ele..."
3,https://theantimedia.com/israel-shoot-to-kill-...,9-Oct-18,Israel’s Education Minister Wants “Shoot to Ki...,"('MEMO', 'https://www.middleeastmonitor.com/20...","b""(MEMO)\xc2\xa0\xe2\x80\x94\xc2\xa0Israel\xe2...","[israel, education, minister, naftali, bennett...","[[israel, education, minister, naftali, bennet..."
4,https://theantimedia.com/nikki-haley-resigns-u...,9-Oct-18,Nikki Haley Resigns as UN Ambassador,"('ZHE', 'https://www.zerohedge.com/news/2018-1...",b'(ZHE)\xc2\xa0\xe2\x80\x94\xc2\xa0 Update (11...,"[update, 11:00, et, joint, press, conference, ...","[[update, 11:00, et], [joint, press, conferenc..."


In [0]:
tokens_list=list(censored_df['tokens'])
for tokens in tokens_list:
  if "'s" in tokens:
    print(tokens)

In [0]:
drive.flush_and_unmount()